In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import torch.nn as nn
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights

In [ ]:
# Paths
csv_path = "/Users/yuhsuanko/Desktop/UChicago/UChicago_Q3/ML_II/Final_Project/BMI/Data/data.csv"            
img_dir = "/Users/yuhsuanko/Desktop/UChicago/UChicago_Q3/ML_II/Final_Project/BMI/Data/Images"                 
model_path = "/Users/yuhsuanko/Desktop/UChicago/UChicago_Q3/ML_II/Final_Project/best_bmi_model_no_sex.pt"        

In [ ]:
# BMI normalization parameters from training set
mean_bmi = 32.53
std_bmi = 8.04

In [ ]:
# Load test data
df = pd.read_csv(csv_path)
df = df[df["is_training"] == 0].reset_index(drop=True)
df["bmi_z"] = (df["bmi"] - mean_bmi) / std_bmi

In [ ]:
# Image transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
class BMIDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.data = dataframe
        self.img_dir = img_dir
        self.transform = transform

        valid_indices = []
        for idx in range(len(self.data)):
            img_path = os.path.join(self.img_dir, self.data.loc[idx, "name"].strip())
            if os.path.exists(img_path):
                valid_indices.append(idx)
        self.valid_indices = valid_indices

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        real_idx = self.valid_indices[idx]
        img_path = os.path.join(self.img_dir, self.data.loc[real_idx, "name"].strip())
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        bmi_z = self.data.loc[real_idx, "bmi_z"]
        return image, torch.tensor(bmi_z, dtype=torch.float32)


In [ ]:
# DataLoader
test_dataset = BMIDataset(df, img_dir, transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# Model architecture
class EfficientNetBMI_NoSex(nn.Module):
    def __init__(self):
        super().__init__()
        weights = EfficientNet_V2_S_Weights.DEFAULT
        self.backbone = efficientnet_v2_s(weights=weights)
        num_features = self.backbone.classifier[1].in_features
        self.backbone.classifier = nn.Identity()

        self.fc = nn.Sequential(
            nn.BatchNorm1d(num_features),
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        features = self.backbone(x)
        return self.fc(features)


In [ ]:
# Load model
from torchvision.models import efficientnet_b0

# Load features
base_model = efficientnet_b0(pretrained=False)
backbone = torch.nn.Sequential(*list(base_model.children())[:-1], torch.nn.Flatten())

# regressor
model = EfficientNetBMI_NoSex()
model.load_state_dict(torch.load(model_path, map_location="cpu"))
model.eval()

/opt/anaconda3/envs/adsp-nlp/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/adsp-nlp/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


EfficientNetBMI_NoSex(
  (backbone): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        )
        (1): FusedMBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (1): B

In [ ]:
# Predict on test set
y_true, y_pred = [], []
with torch.no_grad():
    for images, targets in test_loader:
        images = images
        outputs = model(images).cpu().numpy()
        preds = outputs * std_bmi + mean_bmi  # Denormalize BMI
        actuals = targets.numpy() * std_bmi + mean_bmi
        y_pred.extend(preds)
        y_true.extend(actuals)

In [ ]:
# Evaluation Metrics
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")

MAE: 5.40
RMSE: 7.73
R²: 0.2958
